In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import cross_val_score
from sklearn.svm import LinearSVR
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import confusion_matrix

In [2]:
csv_path = os.path.join("..", "..", "data","LINKED_DATA", "TSR_EHR", "TSR_6_CLEANED.csv")
tsr_6 = pd.read_csv(csv_path)
tsr_6.head()

,height_nm,weight_nm,edu_id,pro_id,opc_id,ih_fl,ivtpamg_nm,hospitalised_time,nivtpa_id,nivtpa1_fl,...,nihs_8_out,nihs_9_out,nihs_10_out,nihs_11_out,total_out,SexName,Age,mrs_tx_1,mrs_tx_3,mrs_tx_6
0,153.0,62.0,3,1,3,0,0.0,8.0,0,999,...,1,0,1,0,4,0,67.0,1,1,1
1,152.0,62.0,3,1,2,0,0.0,4.0,0,999,...,1,0,0,0,1,0,69.0,1,0,0
2,148.0,56.0,2,1,2,0,0.0,5.0,0,999,...,1,0,0,0,2,0,71.0,0,0,0
3,152.0,56.0,4,1,2,0,0.0,3.0,1,0,...,0,0,0,0,0,0,71.0,0,0,0
4,160.0,60.0,2,1,3,0,0.0,4.0,0,999,...,0,0,0,0,4,0,62.0,3,3,3


In [3]:
tsr_6_input = tsr_6.drop(["mrs_tx_6"], axis=1)
tsr_6_input[tsr_6_input == "N"] = 0
tsr_6_input[tsr_6_input == "Y"] = 1
tsr_6_input = tsr_6_input.astype("float64")
tsr_6_input = np.array(tsr_6_input.values)

tsr_6_input_nomrs = tsr_6.drop(["mrs_tx_6", "mrs_tx_3", "mrs_tx_1"], axis=1)
tsr_6_input_nomrs[tsr_6_input_nomrs == "N"] = 0
tsr_6_input_nomrs[tsr_6_input_nomrs == "Y"] = 1
tsr_6_input_nomrs = tsr_6_input_nomrs.astype("float64")
tsr_6_input_nomrs = np.array(tsr_6_input_nomrs.values)

# 6 classes

In [4]:
tsr_6_output = tsr_6.mrs_tx_1
tsr_6_output = tsr_6_output.astype("float64")
tsr_6_output = np.array(tsr_6_output.values)

## SVM

In [5]:
svr = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores = cross_val_score(svr,tsr_6_input,tsr_6_output,cv = 10, scoring='r2')
print(svr_scores)
print("Mean of R^2:", svr_scores.mean())
print("Std of R^2:", svr_scores.std())

[0.94558024 0.97235165 0.96707662 0.98024068 0.98257862 0.92764377
 0.9707805  0.98994036 0.97451639 0.98035596]
Mean of R^2: 0.9691064804445538
Std of R^2: 0.017847778671996863


In [6]:
svr.fit(tsr_6_input,tsr_6_output)
svr_predict =svr.predict(tsr_6_input)
print(svr_predict)
svr_predict = np.round(svr_predict)
print(svr_predict)
print("Accuracy of RF:", (tsr_6_output == svr_predict).sum() / len(svr_predict))

[0.99671584 0.77933688 0.24873182 ... 4.91792978 0.70466806 1.08830069]
[1. 1. 0. ... 5. 1. 1.]
Accuracy of RF: 0.9132739099012286


In [7]:
svr_pred = cross_val_predict(svr,tsr_6_input,tsr_6_output,cv = 10)
svr_pred = np.round(svr_pred)
confusion_matrix(tsr_6_output, svr_pred)

array([[   0,    0,    0,    0,    0,    0,    0],
       [   0,  401,   64,    1,    0,    0,    0],
       [   2,    1, 1028,   36,    0,    0,    0],
       [   0,    0,   24,  635,   24,    0,    0],
       [   0,    0,    0,   37,  550,    9,    0],
       [   0,    0,    0,    0,   52,  749,   15],
       [   0,    0,    0,    1,    2,   35,  485]], dtype=int64)

In [8]:
svr_pred[svr_pred == -1] = 0
confusion_matrix(tsr_6_output, svr_pred)

array([[ 401,   64,    1,    0,    0,    0],
       [   3, 1028,   36,    0,    0,    0],
       [   0,   24,  635,   24,    0,    0],
       [   0,    0,   37,  550,    9,    0],
       [   0,    0,    0,   52,  749,   15],
       [   0,    0,    1,    2,   35,  485]], dtype=int64)

In [9]:
svr1 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores1 = cross_val_score(svr1,tsr_6_input_nomrs,tsr_6_output,cv = 10, scoring='r2')
print(svr_scores1)
print("Mean of R^2:", svr_scores1.mean())
print("Std of R^2:", svr_scores1.std())

[0.70965213 0.7988666  0.82486122 0.81099092 0.81424815 0.78983829
 0.83270923 0.75238452 0.76126461 0.66301644]
Mean of R^2: 0.7757832104157486
Std of R^2: 0.05191020894721774


In [10]:
svr1.fit(tsr_6_input_nomrs,tsr_6_output)
svr_predict1 =svr1.predict(tsr_6_input_nomrs)
print(svr_predict1)
svr_predict1 = np.round(svr_predict1)
print(svr_predict1)
print("Accuracy of RF:", (tsr_6_output == svr_predict1).sum() / len(svr_predict1))

[1.09454561 0.84411792 0.92634792 ... 4.85452348 1.77608397 1.80991193]
[1. 1. 1. ... 5. 2. 2.]
Accuracy of RF: 0.5504697663213683


In [11]:
svr_pred1 = cross_val_predict(svr1,tsr_6_input_nomrs,tsr_6_output,cv = 10)
svr_pred1 = np.round(svr_pred1)
confusion_matrix(tsr_6_output, svr_pred1)

array([[ 61, 336,  59,   9,   0,   1,   0],
       [ 13, 818, 199,  28,   8,   1,   0],
       [  3, 278, 304,  87,  11,   0,   0],
       [  0,  20, 171, 337,  62,   6,   0],
       [  0,   6,  30, 212, 463, 105,   0],
       [  0,   5,   6,  24, 148, 329,  11],
       [  0,   0,   0,   0,   0,   0,   0]], dtype=int64)

In [12]:
svr_pred1[svr_pred1 == 6] = 5
confusion_matrix(tsr_6_output, svr_pred1)

array([[ 61, 336,  59,   9,   0,   1],
       [ 13, 818, 199,  28,   8,   1],
       [  3, 278, 304,  87,  11,   0],
       [  0,  20, 171, 337,  62,   6],
       [  0,   6,  30, 212, 463, 105],
       [  0,   5,   6,  24, 148, 340]], dtype=int64)

## RF

In [13]:
rfr = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores = cross_val_score(rfr,tsr_6_input,tsr_6_output,cv = 10, scoring='r2')
print(rfr_scores)
print("Mean of R^2:", rfr_scores.mean())
print("Std of R^2:", rfr_scores.std())

[0.99996182 0.99996745 0.99998849 0.99991848 0.9999752  0.99996659
 0.99991892 0.99985967 0.99998559 0.99999551]
Mean of R^2: 0.9999537729717906
Std of R^2: 4.0187223876986936e-05


In [14]:
rfr.fit(tsr_6_input,tsr_6_output)
rfr_predict =rfr.predict(tsr_6_input)
print(rfr_predict)
rfr_predict = np.round(rfr_predict)
print(rfr_predict)
print("Accuracy of RF:", (tsr_6_output == rfr_predict).sum() / len(rfr_predict))

[1. 1. 0. ... 5. 0. 1.]
[1. 1. 0. ... 5. 0. 1.]
Accuracy of RF: 1.0


In [15]:
rfr_pred = cross_val_predict(rfr,tsr_6_input,tsr_6_output,cv = 10)
rfr_pred = np.round(rfr_pred)
confusion_matrix(tsr_6_output, rfr_pred)

array([[ 466,    0,    0,    0,    0,    0],
       [   0, 1067,    0,    0,    0,    0],
       [   0,    0,  683,    0,    0,    0],
       [   0,    0,    0,  596,    0,    0],
       [   0,    0,    0,    0,  816,    0],
       [   0,    0,    0,    0,    0,  523]], dtype=int64)

In [16]:
rfr1 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores1 = cross_val_score(rfr1,tsr_6_input_nomrs,tsr_6_output,cv = 10, scoring='r2')
print(rfr_scores1)
print("Mean of R^2:", rfr_scores1.mean())
print("Std of R^2:", rfr_scores1.std())

[0.78461641 0.83812407 0.85364225 0.80021598 0.83401323 0.82829245
 0.88028419 0.75923783 0.8452423  0.79136409]
Mean of R^2: 0.8215032792722285
Std of R^2: 0.03482763312237961


In [17]:
rfr1.fit(tsr_6_input_nomrs,tsr_6_output)
rfr_predict1 =rfr1.predict(tsr_6_input_nomrs)
print(rfr_predict1)
rfr_predict1 = np.round(rfr_predict1)
print(rfr_predict1)
print("Accuracy of RF:", (tsr_6_output == rfr_predict1).sum() / len(rfr_predict1))

[0.93333333 0.93333333 0.         ... 4.86666667 0.26666667 1.06666667]
[1. 1. 0. ... 5. 0. 1.]
Accuracy of RF: 0.9347145266200916


In [18]:
rfr_pred1 = cross_val_predict(rfr1,tsr_6_input_nomrs,tsr_6_output,cv = 10)
rfr_pred1 = np.round(rfr_pred1)
confusion_matrix(tsr_6_output, rfr_pred1)

array([[282, 140,  26,  17,   0,   1],
       [ 54, 757, 205,  44,   5,   2],
       [  2, 132, 435, 102,  12,   0],
       [  0,  14, 130, 378,  72,   2],
       [  0,   7,  16, 149, 599,  45],
       [  0,   3,   2,  12, 146, 360]], dtype=int64)

## XGBoost

In [19]:
xgbr = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores = cross_val_score(xgbr,tsr_6_input,tsr_6_output,cv = 10, scoring='r2')
print(xgbr_scores)
print("Mean of R^2:", xgbr_scores.mean())
print("Std of R^2:", xgbr_scores.std())

[0.99993897 0.99995103 0.99994665 0.99995117 0.99994241 0.99994807
 0.99994522 0.99994435 0.99993649 0.99991847]
Mean of R^2: 0.999942283896733
Std of R^2: 9.123852646233461e-06


In [20]:
xgbr.fit(tsr_6_input,tsr_6_output)
xgbr_predict =xgbr.predict(tsr_6_input)
print(xgbr_predict)
xgbr_predict = np.round(xgbr_predict)
print(xgbr_predict)
print("Accuracy of RF:", (tsr_6_output == xgbr_predict).sum() / len(xgbr_predict))

[9.9761188e-01 9.9761188e-01 2.4066679e-03 ... 4.9783726e+00 2.4066679e-03
 9.9761188e-01]
[1. 1. 0. ... 5. 0. 1.]
Accuracy of RF: 1.0


In [21]:
xgbr_pred = cross_val_predict(xgbr,tsr_6_input,tsr_6_output,cv = 10)
xgbr_pred = np.round(xgbr_pred)
confusion_matrix(tsr_6_output, xgbr_pred)

array([[ 466,    0,    0,    0,    0,    0],
       [   0, 1067,    0,    0,    0,    0],
       [   0,    0,  683,    0,    0,    0],
       [   0,    0,    0,  596,    0,    0],
       [   0,    0,    0,    0,  816,    0],
       [   0,    0,    0,    0,    0,  523]], dtype=int64)

In [22]:
xgbr1 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
xgbr_scores1 = cross_val_score(xgbr1,tsr_6_input_nomrs,tsr_6_output,cv = 10, scoring='r2')
print(xgbr_scores1)
print("Mean of R^2:", xgbr_scores1.mean())
print("Std of R^2:", xgbr_scores1.std())

[0.78461641 0.83812407 0.85364225 0.80021598 0.83401323 0.82829245
 0.88028419 0.75923783 0.8452423  0.79136409]
Mean of R^2: 0.8215032792722285
Std of R^2: 0.03482763312237961


In [23]:
xgbr1.fit(tsr_6_input_nomrs,tsr_6_output)
xgbr_predict1 =xgbr1.predict(tsr_6_input_nomrs)
print(xgbr_predict1)
xgbr_predict1 = np.round(xgbr_predict1)
print(xgbr_predict1)
print("Accuracy of RF:", (tsr_6_output == xgbr_predict1).sum() / len(xgbr_predict1))

[0.93333333 0.93333333 0.         ... 4.86666667 0.26666667 1.06666667]
[1. 1. 0. ... 5. 0. 1.]
Accuracy of RF: 0.9347145266200916


In [24]:
xgbr_pred1 = cross_val_predict(xgbr1,tsr_6_input_nomrs,tsr_6_output,cv = 10)
xgbr_pred1 = np.round(xgbr_pred1)
confusion_matrix(tsr_6_output, xgbr_pred1)

array([[282, 140,  26,  17,   0,   1],
       [ 54, 757, 205,  44,   5,   2],
       [  2, 132, 435, 102,  12,   0],
       [  0,  14, 130, 378,  72,   2],
       [  0,   7,  16, 149, 599,  45],
       [  0,   3,   2,  12, 146, 360]], dtype=int64)

# 2 classes

In [25]:
tsr_6_output[(tsr_6_output == 0)|(tsr_6_output == 1)|(tsr_6_output == 2)] = 0
tsr_6_output[(tsr_6_output == 3)|(tsr_6_output == 4)|(tsr_6_output == 5)] = 1

## SVM

In [26]:
svr2 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores2 = cross_val_score(svr2,tsr_6_input,tsr_6_output,cv = 10, scoring='r2')
print(svr_scores2)
print("Mean of R^2:", svr_scores2.mean())
print("Std of R^2:", svr_scores2.std())

[0.75130125 0.80607881 0.8214153  0.85036427 0.82604788 0.82589893
 0.78090557 0.76925601 0.80772913 0.70966276]
Mean of R^2: 0.7948659905846158
Std of R^2: 0.04002462622467586


In [27]:
svr2.fit(tsr_6_input,tsr_6_output)
svr_predict2 =svr2.predict(tsr_6_input)
print(svr_predict2)
svr_predict2 = np.round(svr_predict2)
print(svr_predict2)
print("Accuracy of RF:", (tsr_6_output == svr_predict2).sum() / len(svr_predict2))

[-0.01423756 -0.0606813  -0.22220943 ...  1.26552534 -0.1969139
  0.03872127]
[-0. -0. -0. ...  1. -0.  0.]
Accuracy of RF: 0.9828956877860756


In [28]:
svr_pred2 = cross_val_predict(svr2,tsr_6_input,tsr_6_output,cv = 10)
svr_pred2 = np.round(svr_pred2)
confusion_matrix(tsr_6_output, svr_pred2)

array([[   0,    0,    0,    0],
       [   3, 2138,   75,    0],
       [   0,   70, 1863,    2],
       [   0,    0,    0,    0]], dtype=int64)

In [29]:
svr_pred2[svr_pred2 == -1] = 0
svr_pred2[svr_pred2 == 2] = 1
confusion_matrix(tsr_6_output, svr_pred2)

array([[2141,   75],
       [  70, 1865]], dtype=int64)

In [30]:
svr3 = LinearSVR(epsilon = 0, dual=False, loss = "squared_epsilon_insensitive", C = 1, random_state = 19)
svr_scores3 = cross_val_score(svr3,tsr_6_input_nomrs,tsr_6_output,cv = 10, scoring='r2')
print(svr_scores3)
print("Mean of R^2:", svr_scores3.mean())
print("Std of R^2:", svr_scores3.std())

[0.57735334 0.67568588 0.72407621 0.74087481 0.7417242  0.70845081
 0.71058757 0.66596272 0.73495655 0.55116464]
Mean of R^2: 0.6830836727824314
Std of R^2: 0.06440423545737073


In [31]:
svr3.fit(tsr_6_input_nomrs,tsr_6_output)
svr_predict3 =svr3.predict(tsr_6_input_nomrs)
print(svr_predict3)
svr_predict3 = np.round(svr_predict3)
print(svr_predict3)
print("Accuracy of RF:", (tsr_6_output == svr_predict3).sum() / len(svr_predict3))

[ 0.01445872 -0.02641789  0.0041937  ...  1.2330879   0.20351796
  0.1103934 ]
[ 0. -0.  0. ...  1.  0.  0.]
Accuracy of RF: 0.9159238737653578


In [32]:
svr_pred3 = cross_val_predict(svr3,tsr_6_input_nomrs,tsr_6_output,cv = 10)
svr_pred3 = np.round(svr_pred3)
confusion_matrix(tsr_6_output, svr_pred3)

array([[2036,  180],
       [ 212, 1723]], dtype=int64)

## RF

In [33]:
rfr2 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores2 = cross_val_score(rfr2,tsr_6_input,tsr_6_output,cv = 10, scoring='r2')
print(rfr_scores2)
print("Mean of R^2:", rfr_scores2.mean())
print("Std of R^2:", rfr_scores2.std())

[1.         1.         1.         0.99986738 1.         1.
 1.         1.         1.         1.        ]
Mean of R^2: 0.9999867382481705
Std of R^2: 3.9785255488455905e-05


In [34]:
rfr2.fit(tsr_6_input,tsr_6_output)
rfr_predict2 =rfr2.predict(tsr_6_input)
print(rfr_predict2)
rfr_predict2 = np.round(rfr_predict2)
print(rfr_predict2)
print("Accuracy of RF:", (tsr_6_output == rfr_predict2).sum() / len(rfr_predict2))

[0. 0. 0. ... 1. 0. 0.]
[0. 0. 0. ... 1. 0. 0.]
Accuracy of RF: 1.0


In [35]:
rf_pred2 = cross_val_predict(rfr2,tsr_6_input,tsr_6_output,cv = 10)
rf_pred2 = np.round(rf_pred2)
confusion_matrix(tsr_6_output, rf_pred2)

array([[2216,    0],
       [   0, 1935]], dtype=int64)

In [36]:
rfr3 = RandomForestRegressor(criterion = "mse", n_estimators = 15, bootstrap=True, random_state = 19,max_features = 0.8)
rfr_scores3 = cross_val_score(rfr3,tsr_6_input_nomrs,tsr_6_output,cv = 10, scoring='r2')
print(rfr_scores3)
print("Mean of R^2:", rfr_scores3.mean())
print("Std of R^2:", rfr_scores3.std())

[0.61955993 0.74186447 0.83176085 0.76283567 0.78815204 0.73151756
 0.75826261 0.72771581 0.82543574 0.62756759]
Mean of R^2: 0.7414672274832117
Std of R^2: 0.06800337265622568


In [37]:
rfr3.fit(tsr_6_input_nomrs,tsr_6_output)
rfr_predict3 =rfr3.predict(tsr_6_input_nomrs)
print(rfr_predict3)
rfr_predict3 = np.round(rfr_predict3)
print(rfr_predict3)
print("Accuracy of RF:", (tsr_6_output == rfr_predict3).sum() / len(rfr_predict3))

[0.06666667 0.         0.         ... 1.         0.         0.06666667]
[0. 0. 0. ... 1. 0. 0.]
Accuracy of RF: 0.9968682245242111


In [38]:
rfr_pred3 = cross_val_predict(rfr3,tsr_6_input_nomrs,tsr_6_output,cv = 10)
rfr_pred3 = np.round(rfr_pred3)
confusion_matrix(tsr_6_output, rfr_pred3)

array([[2009,  207],
       [ 121, 1814]], dtype=int64)

## XGBoost

In [39]:
xgbr2 = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores2 = cross_val_score(xgbr2,tsr_6_input,tsr_6_output,cv = 10, scoring='r2')
print(xgbr_scores2)
print("Mean of R^2:", xgbr_scores2.mean())
print("Std of R^2:", xgbr_scores2.std())

[0.99997719 0.99997704 0.99997708 0.99997658 0.99997729 0.99997701
 0.99997684 0.9999771  0.99997714 0.99997599]
Mean of R^2: 0.999976926155173
Std of R^2: 3.6484455271521257e-07


In [40]:
xgbr2.fit(tsr_6_input,tsr_6_output)
xgbr_predict2 =xgbr2.predict(tsr_6_input)
print(xgbr_predict2)
xgbr_predict2 = np.round(xgbr_predict2)
print(xgbr_predict2)
print("Accuracy of RF:", (tsr_6_output == xgbr_predict2).sum() / len(xgbr_predict2))

[0.00238067 0.00238067 0.00238067 ... 0.9976183  0.00238067 0.00238067]
[0. 0. 0. ... 1. 0. 0.]
Accuracy of RF: 1.0


In [41]:
xgbr_pred2 = cross_val_predict(xgbr2,tsr_6_input,tsr_6_output,cv = 10)
xgbr_pred2 = np.round(xgbr_pred2)
confusion_matrix(tsr_6_output, xgbr_pred2)

array([[2216,    0],
       [   0, 1935]], dtype=int64)

In [42]:
xgbr3 = XGBRegressor(n_estimators  = 15, objective="reg:squarederror", random_state = 19) 
xgbr_scores3 = cross_val_score(xgbr3,tsr_6_input_nomrs,tsr_6_output,cv = 10, scoring='r2')
print(xgbr_scores3)
print("Mean of R^2:", xgbr_scores3.mean())
print("Std of R^2:", xgbr_scores3.std())

[0.66684919 0.7667298  0.81267796 0.76909761 0.77169132 0.7166649
 0.78748053 0.68365637 0.83661724 0.61305075]
Mean of R^2: 0.7424515666316938
Std of R^2: 0.06675556623211351


In [43]:
xgbr3.fit(tsr_6_input_nomrs,tsr_6_output)
xgbr_predict3 =xgbr3.predict(tsr_6_input_nomrs)
print(xgbr_predict3)
xgbr_predict3 = np.round(xgbr_predict3)
print(xgbr_predict3)
print("Accuracy of RF:", (tsr_6_output == xgbr_predict3).sum() / len(xgbr_predict3))

[ 0.00560121 -0.00831811  0.00754873 ...  0.99837583  0.02824771
  0.01723414]
[ 0. -0.  0. ...  1.  0.  0.]
Accuracy of RF: 0.9831365935919055


In [44]:
xgbr_pred3 = cross_val_predict(xgbr3,tsr_6_input_nomrs,tsr_6_output,cv = 10)
xgbr_pred3 = np.round(xgbr_pred3)
confusion_matrix(tsr_6_output, xgbr_pred3)

array([[2008,  208],
       [ 118, 1817]], dtype=int64)

# Summary

## Mean & Std

In [45]:
svr_mean = np.array([svr_scores.mean(), svr_scores.std(), svr_scores[0], svr_scores[1], svr_scores[2], svr_scores[3],
                     svr_scores[4], svr_scores[5], svr_scores[6], svr_scores[7], svr_scores[8], svr_scores[9]])
rfr_mean = np.array([rfr_scores.mean(), rfr_scores.std(), rfr_scores[0], rfr_scores[1], rfr_scores[2], rfr_scores[3],rfr_scores[4], 
                    rfr_scores[5], rfr_scores[6], rfr_scores[7], rfr_scores[8], rfr_scores[9]])
xgbr_mean = np.array([xgbr_scores.mean(), xgbr_scores.std(), xgbr_scores[0], xgbr_scores[1], xgbr_scores[2], xgbr_scores[3],
                     xgbr_scores[4], xgbr_scores[5], xgbr_scores[6], xgbr_scores[7], xgbr_scores[8], xgbr_scores[9]])
svr_mean2 = np.array([svr_scores2.mean(), svr_scores2.std(), svr_scores2[0], svr_scores2[1], svr_scores2[2], svr_scores2[3],
                     svr_scores2[4], svr_scores2[5], svr_scores2[6], svr_scores2[7], svr_scores2[8], svr_scores2[9]])
rfr_mean2 = np.array([rfr_scores2.mean(), rfr_scores2.std(), rfr_scores2[0], rfr_scores2[1], rfr_scores2[2], rfr_scores2[3],rfr_scores2[4], 
                    rfr_scores2[5], rfr_scores2[6], rfr_scores2[7], rfr_scores2[8], rfr_scores2[9]])
xgbr_mean2 = np.array([xgbr_scores2.mean(), xgbr_scores2.std(), xgbr_scores2[0], xgbr_scores2[1], xgbr_scores2[2], xgbr_scores2[3],
                     xgbr_scores2[4], xgbr_scores2[5], xgbr_scores2[6], xgbr_scores2[7], xgbr_scores2[8], xgbr_scores2[9]])

In [46]:
tsr_6_mean = pd.DataFrame([svr_mean, rfr_mean, xgbr_mean, svr_mean2, rfr_mean2, xgbr_mean2]).T
tsr_6_mean.index = ["Mean", "Std", "R^2_1", "R^2_2", "R^2_3", "R^2_4", "R^2_5", "R^2_6", "R^2_7", "R^2_8", "R^2_9", "R^2_10"]
tsr_6_mean.columns = ["svr", 'rfr', 'xgbr', 'svr2', 'rfr2', 'xgbr2']

In [47]:
csv_save = os.path.join(".", "tsr_6_mean_regression.csv")
tsr_6_mean.to_csv(csv_save, index = True)